# Ultimate Inc

## Part 1 of Data Science Challenge

The attached logins.json file contains (simulated) timestamps of user logins in a particular geographic location.
Aggregate these login counts based on 15-minute time intervals, and visualize and describe the resulting time series of login counts in ways that best characterize the underlying patterns of the demand.
Please report/illustrate important features of the demand, such as daily cycles. 
If there are data quality issues, please report them.

In [44]:
import pandas as pd

In [45]:
df = pd.read_json('logins.json')
df.head(3)

,login_time
0,1970-01-01 20:13:18
1,1970-01-01 20:16:10
2,1970-01-01 20:16:37


### Wrangling Data

Will be working with data that will require time series analysis.
In some cases I'd reach out to the interviewer (or business analyst) to see if a specific time series library was preferred by the company/team.
In this case, assuming the challenge is expected to be solved without using these types of libraries.

In [46]:
# Has 1 column and 90k+ rows.
df.shape

(93142, 1)

In [47]:
# Checking for missing values
df.isnull().sum()

login_time    0
dtype: int64

In [48]:
# Converting to a datetime format
df['login_time'] = pd.to_datetime(df['login_time'])
df.sort_values('login_time')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93142 entries, 0 to 93141
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   login_time  93142 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 727.8 KB


In [49]:
# Checking to see if login_time is unique so it can be used as the index.
df['login_time'].nunique()

92265

877 are NOT unique (less than 1%).

Since this is simulated data will drop these values and then set the index to the login_time.
However, may not do this on a real project because these could be different users that are logged in at the exact same time because we only have date times that go to the second.

In [50]:
df.drop_duplicates(inplace=True)
df.shape

(92265, 1)

In [51]:
# Checking time ranges.

print(f"First date: {df['login_time'].min()}.")
print(f"Last date: {df['login_time'].max()}.")

First date: 1970-01-01 20:12:16.
Last date: 1970-04-13 18:57:38.


In [52]:
# Create new columns
# This default value will change in calculations later.
df['login_count'] = 1
# Separating out the various time components by referencing https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-components
df['year'] = df['login_time'].dt.year
df['month'] = df['login_time'].dt.month
df['day'] = df['login_time'].dt.day
df['hour'] = df['login_time'].dt.hour
df['minute'] = df['login_time'].dt.minute
df['second'] = df['login_time'].dt.second
# Monday=0, Sunday=6
df['day_of_week'] = df['login_time'].dt.day_of_week
# Changed pattern because weekofyear is deprecated
df['week_of_year'] = df['login_time'].dt.isocalendar().week
df['day_of_year'] = df['login_time'].dt.day_of_year
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92265 entries, 0 to 93141
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   login_time    92265 non-null  datetime64[ns]
 1   login_count   92265 non-null  int64         
 2   year          92265 non-null  int64         
 3   month         92265 non-null  int64         
 4   day           92265 non-null  int64         
 5   hour          92265 non-null  int64         
 6   minute        92265 non-null  int64         
 7   second        92265 non-null  int64         
 8   day_of_week   92265 non-null  int64         
 9   week_of_year  92265 non-null  UInt32        
 10  day_of_year   92265 non-null  int64         
dtypes: UInt32(1), datetime64[ns](1), int64(9)
memory usage: 8.2 MB


In [53]:
df = df.set_index('login_time')
df.value_counts()

login_count  year  month  day  hour  minute  second  day_of_week  week_of_year  day_of_year
1            1970  1      1    20    12      16      3            1             1              1
                   3      16   3     19      5       0            12            75             1
                                     11      51      0            12            75             1
                                     9       54      0            12            75             1
                                             50      0            12            75             1
                                                                                              ..
                   2      13   19    41      59      4            7             44             1
                                     39      13      4            7             44             1
                                     38      33      4            7             44             1
                                   

In [54]:
# Create new dataframe for grouping by 15 minutes.
df_15_min = pd.DataFrame(df['login_count'].resample('15Min').sum())
df_15_min['year'] = df_15_min.index.year
df_15_min['month'] = df_15_min.index.month
df_15_min['day'] = df_15_min.index.day
df_15_min['hour'] = df_15_min.index.hour
df_15_min['minute'] = df_15_min.index.minute
# Monday=0, Sunday=6
df_15_min['day_of_week'] = df_15_min.index.day_of_week
# Changed pattern because weekofyear is deprecated
df_15_min['week_of_year'] = df_15_min.index.isocalendar().week
df_15_min['day_of_year'] = df_15_min.index.day_of_year
df_15_min.head()

,login_count,year,month,day,hour,minute,day_of_week,week_of_year,day_of_year
login_time,,,,,,,,,
1970-01-01 20:00:00,2,1970,1,1,20,0,3,1,1
1970-01-01 20:15:00,6,1970,1,1,20,15,3,1,1
1970-01-01 20:30:00,9,1970,1,1,20,30,3,1,1
1970-01-01 20:45:00,7,1970,1,1,20,45,3,1,1
1970-01-01 21:00:00,1,1970,1,1,21,0,3,1,1


In [55]:
df_15_min.value_counts()

login_count  year  month  day  hour  minute  day_of_week  week_of_year  day_of_year
0            1970  1      1    21    30      3            1             1              1
11           1970  2      7    19    45      5            6             38             1
                          6    3     45      4            6             37             1
                               12    45      4            6             37             1
                               19    30      4            6             37             1
                                                                                      ..
5            1970  1      11   23    45      6            2             11             1
                          12   1     0       0            3             12             1
                               4     15      0            3             12             1
                                     45      0            3             12             1
68           1970  3      

### Visualize Data